In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('bestsellers with categories.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


# Hypoteshis Testing For Median: Non Parametric

Uji hipotesis parametrik memiliki asumsi-asumsi yang perlu dipenuhi, biasanya berupa distribusi data yang normal. Bagaimana apabila asumsi tersebut tidak dapat dipenuhi? Kita bisa mencoba menggunakan Uji Non Parametrik.  

Pada Uji perbandingan non parametrik, kita membandingkan **median** antar populasi (parametrik, yang dibandingkan adalah mean).  
Uji non parametrik tetap memiliki asusmsi yang harus dipenuhi, biasanya berupa:
* sampel harus diambil secara acak dari populasi
* tipe data numerik/ordinal  

Uji non-Parametric untuk Median: Wilcoxon, Mann Whitney, Kruskal Wallis

In [3]:
# Sebelum melakukan uji hipotesis, seharusnya kita melakukan goodness of fit test, 
# untuk memastikan distribusi data kita sesuai dengan uji yang akan kita lakukan.
# uji non parametrik bisa dipakai pada data yang distribusinya normal, 
# hanya saja, uji pamametrik memiliki power yang lebih tinggi, jadi kalau bisa menggunakan uji parametrik, gunakan parametrik

## Mann Withney U
Uji non parametrik untuk membandingkan median 2 populasi independen (mirip T-test 2 samples)  

Kita akan membandingkan harga buku fiksi dan non fiksi

In [5]:
from scipy.stats import shapiro
stats, pval=shapiro(df['Price'])

if pval>0.05:
    print('Distribusi data diasumsikan normal')
else:
    print('Distribusi data diasumsikan tidak normal')
#GOODNESS of FIT test
#karena tidak normal, pakai uji non parametrik

Distribusi data diasumsikan tidak normal


In [6]:
from scipy.stats import mannwhitneyu
mannwhitneyu(df[df['Genre']=='Fiction']['Price'],df[df['Genre']=='Non Fiction']['Price'])

MannwhitneyuResult(statistic=27294.0, pvalue=7.981217771671606e-08)

**Hypothesis**  
Ho: Median dari harga buku non fiksi sama dengan harga buku fiksi  
Ha: Median dari harga buku non fiksi lebih tinggi dari harga buku fiksi (one-tailed)

In [10]:
stats,pval=mannwhitneyu(
    df[df['Genre']=='Non Fiction']['Price'],
    df[df['Genre']=='Fiction']['Price'],
    alternative='greater'
)

print(pval)

if pval<=0.05:
    print('tolak Ho')
else:
    print('terima Ho')

3.990608885835803e-08
tolak Ho


karena p-value lebih kecil hari 0.05, kita punya cukup bukti untuk menolak Ho, artinya harga buku fiksi lebih tinggi dari harga buku non fiksi

## Wilcoxon
Uji non parametric untuk 2 populasi berpasangan (seperti t-test paired)

In [12]:
df_paired=pd.DataFrame({'using smartphone':[604,556,540,522,469,544,513,470,556,531,599,537,619,536,554,467],
 'no smartphone':[636, 623, 615, 672, 601, 600, 542, 554, 543, 520,609, 559, 595, 565, 573, 554]})
df_paired['diff']=df_paired['using smartphone']-df_paired['no smartphone']

df_paired

,using smartphone,no smartphone,diff
0,604,636,-32
1,556,623,-67
2,540,615,-75
3,522,672,-150
4,469,601,-132
5,544,600,-56
6,513,542,-29
7,470,554,-84
8,556,543,13
9,531,520,11


In [14]:
# goodness of fit test
stats, pval=shapiro(df_paired['diff'])

if pval>0.05:
    print('Distribusi data diasumsikan normal')
else:
    print('Distribusi data diasumsikan tidak normal')
    #karena kita mau coba wilcoxon, anggap saja tidak normal

Distribusi data diasumsikan normal


**Hypothesis**

Ho: Median perbedaan kecepatan reaksi sama dengan 0 (pakai atau tidak pakai hp, kecepatan reaksi sama)  
Ha: median perbedaan kecepatan reaksi tidak sama dengan 0 (two tailed) (pakai hp beda kecepatan reaksinya dengan yang tidak pakai)

In [15]:
from scipy.stats import wilcoxon

stats, pval= wilcoxon(df_paired['diff']) #alternative default two sided

print(pval)

if pval<=0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

0.001678466796875
Tolak Ho


Karena P-val lebih kecil 0.05, kita punya cukup bukti untuk menolak Ho, artinya, ada perbedaan kecepatan reaksi antara pengemudi yang memakai smartphone dan tidak.

## Kruskal Wallis
Uji non Parametrik untuk membandingkan median lebih dari 2 populasi (temannya F test/ANOVA one way)  
* ada satu asumsi lagi untuk kruskal wallis, yaitu semua populasi memiliki distribusi yang sama

Kita akan membandingkan harga buku di tahun 2009, 2014 dan 2019  

**Hypothesis**  
Ho: tidak ada perbedaan nilai median harga buku antara tahun 2009, 2014, 2019
H1: setidaknya ada satu pasang di antara tahun 2009, 2014 dan 2019 yang memiliki nilai median yang berbeda. (kita tidak tahu yang mana yang berbeda dan bagaimana perbedaannya)

In [17]:
from scipy.stats import kruskal

In [19]:
stats,pval= kruskal(*[df[df['Year']==i]['Price'] for i in [2009, 2014, 2019]])
print(pval)

if pval<=0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

0.11045396593880222
Terima Ho


In [20]:
# kalau diketik satu2
kruskal(df[df['Year']==2009]['Price'],
       df[df['Year']==2014]['Price'],
       df[df['Year']==2019]['Price'])

KruskalResult(statistic=4.406312885328423, pvalue=0.11045396593880222)

karena P value **tidak** lebih kecil dari 0.05, maka kita **tidak punya cukup bukti** untuk menolak Ho, artinya tidak ada perbedaan harga buku yang signifikan di tahun 2009,2014 dan 2019

# Hypothesis Testing: Proportion
Untuk data kategorikal/qualitatif  

Uji Z untuk populasi tunggal dan 2 populasi, Chi Square

In [56]:
df=pd.read_csv('titanic.csv')

In [22]:
df.head()

,sex,age,parch,fare,class,deck,embark_town,alive,alone
0,male,22.0,0,7.2500,Third,NaN,Southampton,no,False
1,female,38.0,0,71.2833,First,C,Cherbourg,yes,False
2,female,26.0,0,7.9250,Third,NaN,Southampton,yes,True
3,female,35.0,0,53.1000,First,C,Southampton,yes,False
4,male,35.0,0,8.0500,Third,NaN,Southampton,no,True


## Z Test untuk Populasi Tunggal

untuk menguji proporsi dari satu populasi dibandingkan dengan nilai proporsi tertentu.
* Hanya untuk data dikotomi/ binari (yang punya 2 kategori)  

Contoh: kita ingin membandingkan proporsi penumpang wanita dan pria di kapal titanic. Apakah proporsi wanita lebih besar dari pria?

**Hypothesis**  
Ho: proporsi penumpang wanita sama dengan proporsi penumpang pria (proporsi penumpang wanita=0.5)  
H1: proporsi penumpang wanita lebih besar dari penumpang pria (proporsi penumpang wanita > 0.5)

In [23]:
from statsmodels.stats.proportion import proportions_ztest

In [28]:
# 2 argumen yang perlu dimasukan ke proportions_ztest: jumlah penumpang wanita, dan jumlah total data

n_wanita=(df['sex']=='female').sum() # jumlah penumpang wanita
n= len(df['sex']) # total jumlah penumpang

In [29]:
proportions_ztest(n_wanita,n,0.5,'larger') 
# jumlah data yang ingin dibandingkan proporsinya, jumlah total data, proporsi pembanding
# alternative: two sided, smaller, larger. Karena lebih besar, maka kita pakai larger

(-9.22171878272267, 1.0)

In [30]:
stats, pval=proportions_ztest(n_wanita,n,0.5,'larger') 
if pval<=0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

Terima Ho


Karena p val lebih besar dari 0.05, kita tidak punya cukup bukti untuk menolak Ho, artinya, proporsi penumpang wanita tidak lebih besar dari penumpang pria.

In [31]:
n_wanita/n # proporsi wanita <0.5

0.35241301907968575

## Z test untuk 2 populasi
untuk menguji proporsi dari 2 populasi, di mana isi dari kedua populasi berupa variabel kategorikal dikotomi/ binari

contoh: apakah proporsi **penumpang wanita yang selamat** lebih besar dari proporsi **penumpang pria yang selamat**

**Hypothesis**  
Ho: Proporsi wanita yang selamat sama dengan proporsi pria yang selamat  
Ha: Proporsi wanita yang selamat lebih besar dari proporsi pria yang selamat

In [39]:
# kita perlu mencari dulu jumlah penumpang yang selamat berdasarkan gender dan ratio penumpang pria dan wanita

df_count=pd.crosstab(df['sex'],df['alive'])
df_count

alive,no,yes
sex,,
female,81,233
male,468,109


In [36]:
df_count['yes'] #jumlah penumpang yang selamat berdasarkan gender

sex
female    233
male      109
Name: yes, dtype: int64

In [38]:
df_count.sum(axis=1) #jumlah penumpang berdasarkan gender

sex
female    314
male      577
dtype: int64

In [40]:
proportions_ztest(df_count['yes'],df_count.sum(axis=1),alternative='larger') 
#alternative larger karena mau uji penumpang wanita>pria, dan data wanita masuk duluan

(16.218833930670097, 1.8558738850567398e-59)

In [41]:
stats, pval=proportions_ztest(df_count['yes'],df_count.sum(axis=1),alternative='larger')  
if pval<=0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

Tolak Ho


karena P value < 0.05, maka kita punya cukup bukti untuk menolak ho, artinya proporsi penumpang wanita yang selamat lebih besar dari pria

In [42]:
pd.crosstab(df['sex'],df['alive'],normalize='index')

alive,no,yes
sex,,
female,0.257962,0.742038
male,0.811092,0.188908


## Chi square
Untuk menguji apakah dua variabel kategorikal memiliki hubungan satu dengan yang lain. (Chi squared test for independence)  
[Bacaan lanjut](https://towardsdatascience.com/gentle-introduction-to-chi-square-test-for-independence-7182a7414a95)

In [43]:
from scipy.stats import chi2_contingency

In [44]:
df['embark_town'].value_counts()

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64

In [45]:
pd.crosstab(df['embark_town'],df['alive'])

alive,no,yes
embark_town,,
Cherbourg,75,93
Queenstown,47,30
Southampton,427,217


In [46]:
chi2_contingency(pd.crosstab(df['embark_town'],df['alive']))
#output: stats, pval, degree of freedom, expected value

(26.48914983923762,
 1.769922284120912e-06,
 2,
 array([[103.7480315,  64.2519685],
        [ 47.5511811,  29.4488189],
        [397.7007874, 246.2992126]]))

In [59]:
stats,pval, dof, expected=chi2_contingency(pd.crosstab(df['embark_town'],df['alive']))

if pval<=0.05:
    print('Tolak Ho')
else:
    print('Terima Ho')

Tolak Ho


p value lebih kecil dari 0.05, kita punya cukup bukti untuk menolak Ho, artinya ada keterkaikan antara asal kota dengan keselamatan penumpang

Uji perbandingan numerik:  
* parametrik (mean)
    * 1 populasi (T test)
    * 2 populasi independen (T test 2 samples)
    * 2 populasi berpasangan (T test paired)
    * lebih dari 2 populasi (ANOVA one way/F test)  

* Non Parametrik (kalau distibusi tidak normal) (median)
    * 2 populasi independen (Mann Whitney)
    * 2 populasi berpasangan (Wilcoxon)
    * lebih dari 2 populasi (Kruskal Wallis)  
    
Uji perbandingan proporsi (data kategorikal):
   * 1 populasi/variabel berisi 2 kategori: Z test
   * 2 populasi/varibel berisi 2 kategori : Z test
   * 2 variabel dengan >2 kategori: chi square (uji independen)
    

In [60]:
# import seaborn as sns

In [ ]:
# !pip install seaborn